In [1]:
"""
The classic game of flappy bird. Make with python
and pygame. Features pixel perfect collision using masks :o
Date Modified:  Jul 30, 2019
Author: Tech With Tim
Estimated Work Time: 5 hours (1 just for that damn collision)
"""
import pygame
import random
import os
import time
import neat
import visualize
import pickle
import numpy as np
import math

pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
n=3;
global temp_list
temp_list = []
def appen(x1,x2,y):
    global temp_list
    temp_list.append([x1,x2])
    
def appen_training(x1,x2,x3):
    global temp_list
                
    S= max(x1+x2+x3,1.0);
    
    temp_list = []
                                
    appen(x2,x3,S)
    appen(x1,x3,S)
    appen(x1,x2,S)
    
    trainingdata.append(temp_list)
    traininglabel.append(S)
    

def readtrainingdata():
    devided=30
    for i1 in range(devided+1):
        for i2 in range(i1+1):
            for i3 in range(i2+1):
                appen_training(i1/devided,i2/devided,i3/devided);
                            

trainingdata=[]
traininglabel=[]
S=1.0
readtrainingdata();

def appen_test(x1,x2,x3):
    global temp_list
                
    S= max(x1+x2+x3,1.0);
    temp_list = []
                                
    appen(x2,x3,S)
    appen(x1,x3,S)
    appen(x1,x2,S)
    
    testingdata.append(temp_list)
    testinglabel.append(S)
    

def readtestingdata():
    devided=20
    for i1 in range(devided+1):
        for i2 in range(i1+1):
            for i3 in range(i2+1):
                appen_test(i1/devided,i2/devided,i3/devided);
                            

testingdata=[]
testinglabel=[]
S=1.0
readtestingdata();

In [3]:
trainingdata=np.array(trainingdata)
traininglabel=np.array(traininglabel)
testingdata=np.array(testingdata)
testinglabel=np.array(testinglabel)


In [4]:
print(trainingdata)
print(testingdata)

[[[0.         0.        ]
  [0.         0.        ]
  [0.         0.        ]]

 [[0.         0.        ]
  [0.03333333 0.        ]
  [0.03333333 0.        ]]

 [[0.03333333 0.        ]
  [0.03333333 0.        ]
  [0.03333333 0.03333333]]

 ...

 [[1.         0.93333333]
  [1.         0.93333333]
  [1.         1.        ]]

 [[1.         0.96666667]
  [1.         0.96666667]
  [1.         1.        ]]

 [[1.         1.        ]
  [1.         1.        ]
  [1.         1.        ]]]
[[[0.   0.  ]
  [0.   0.  ]
  [0.   0.  ]]

 [[0.   0.  ]
  [0.05 0.  ]
  [0.05 0.  ]]

 [[0.05 0.  ]
  [0.05 0.  ]
  [0.05 0.05]]

 ...

 [[1.   0.9 ]
  [1.   0.9 ]
  [1.   1.  ]]

 [[1.   0.95]
  [1.   0.95]
  [1.   1.  ]]

 [[1.   1.  ]
  [1.   1.  ]
  [1.   1.  ]]]


In [5]:
print(traininglabel)
print(testinglabel)

[1.         1.         1.         ... 2.93333333 2.96666667 3.        ]
[1.   1.   1.   ... 2.9  2.95 3.  ]


In [6]:
gen = 0
def eval_genomes(genomes, config):
    """
    runs the simulation of the current population of
    birds and sets their fitness based on the distance they
    reach in the game.
    """
    global gen
    gen += 1

    # start by creating lists holding the genome itself, the
    # neural network associated with the genome and the
    # bird object that uses that network to play
    nets = []
    individuals = []
    ge = []
    for genome_id, genome in genomes:
        genome.fitness = 0  # start with fitness level of 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        ge.append(genome)

    for net_id in range(len(nets)):
        ge[net_id].fitness = 0
        for data_id in range(len(trainingdata)):
            input_data = trainingdata[data_id]
            input_label = traininglabel[data_id]
            output_list = []

            for temp_id in range(len(input_data)):
                output = nets[net_id].activate(input_data[temp_id])
                output_list.append(output[0])
            #print(output_list, input_label)
            sum_h = sum(output_list)
            #print(sum_h)
            if (n-1)*input_label > sum_h:
                ge[net_id].fitness += -abs((n-1)*input_label -sum_h)
            else:
                ge[net_id].fitness += -(sum_h-(n-1)*input_label)*(sum_h-(n-1)*input_label)
                
                

In [7]:
def run(config_file):
    """
    runs the NEAT algorithm to train a neural network to play flappy bird.
    :param config_file: location of config file
    :return: None
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #p.add_reporter(neat.Checkpointer(5))

    # Run for up to 50 generations.
    winner = p.run(eval_genomes, 50)

    # show final stats
    print('\nBest genome:\n{!s}'.format(winner))


if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    # local_dir = os.path.dirname(__file__)
    local_dir = os.path.dirname(os.path.abspath("__file__"))
    config_path = os.path.join(local_dir, 'config-feedforward.txt')
    run(config_path)


 ****** Running generation 0 ****** 



	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_connection = full_nodirect;
	if not, set initial_connection = full_direct
	if this is desired, set initial_conn

Population's average fitness: -451768.66780 stdev: 836496.25987
Best fitness: -6774.94267 - size: (11, 30) - species 15 - id 15
Average adjusted fitness: 0.872
Mean genetic distance 3.649, standard deviation 0.765
Population of 40 members in 20 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     2  -16913.0    0.997     0
     2    0     2  -940794.9    0.732     0
     3    0     2  -592300.0    0.832     0
     4    0     2  -9854.9    0.999     0
     5    0     2  -16913.0    0.997     0
     6    0     2  -16913.0    0.997     0
     7    0     2  -15582.0    0.997     0
     8    0     2  -162237.7    0.955     0
     9    0     2  -16913.0    0.997     0
    10    0     2  -501121.4    0.858     0
    11    0     2  -323217.5    0.909     0
    12    0     2  -755180.3    0.785     0
    13    0     2  -1908070.0    0.455     0
    14    0     2  -7548.9    1.000     0
    15    0     2  -6774.9    1.000     0
    16    0 

Population's average fitness: -353046.84234 stdev: 1062380.37913
Best fitness: -1819.25691 - size: (11, 29) - species 15 - id 50
Average adjusted fitness: 0.923
Mean genetic distance 2.145, standard deviation 1.529
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15    7    10  -1819.3    0.964     0
    19    7    10  -3475.8    0.882     6
Total extinctions: 0
Generation time: 7.671 sec (13.480 average)

 ****** Running generation 8 ****** 

Population's average fitness: -116804.11028 stdev: 230859.36185
Best fitness: -1819.25691 - size: (11, 29) - species 15 - id 50
Average adjusted fitness: 0.843
Mean genetic distance 2.105, standard deviation 1.504
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15    8    11  -1819.3    0.855     1
    19    8     9  -2221.7    0.832     0
Total extinctions: 0
Generation time: 7.260 sec (12.7

Population's average fitness: -43026.53990 stdev: 119396.49559
Best fitness: -991.93116 - size: (9, 22) - species 15 - id 261
Average adjusted fitness: 0.919
Mean genetic distance 2.192, standard deviation 1.495
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15   23    10   -991.9    0.974     3
    19   23    10  -1071.5    0.864     4
Total extinctions: 0
Generation time: 5.470 sec (5.900 average)

 ****** Running generation 24 ****** 

Population's average fitness: -28362.25414 stdev: 37863.00350
Best fitness: -991.93116 - size: (9, 22) - species 15 - id 261
Average adjusted fitness: 0.829
Mean genetic distance 2.205, standard deviation 1.487
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15   24     9   -991.9    0.775     4
    19   24    11  -1071.5    0.882     5
Total extinctions: 0
Generation time: 5.988 sec (5.867 aver

Population's average fitness: -16535.50315 stdev: 17911.55444
Best fitness: -799.97797 - size: (10, 22) - species 19 - id 352
Average adjusted fitness: 0.766
Mean genetic distance 2.248, standard deviation 1.481
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15   39    10   -991.9    0.743    19
    19   39    10   -800.0    0.788    13
Total extinctions: 0
Generation time: 5.086 sec (5.163 average)

 ****** Running generation 40 ****** 

Population's average fitness: -21208.32421 stdev: 35947.95989
Best fitness: -799.97797 - size: (10, 22) - species 19 - id 352
Average adjusted fitness: 0.878
Mean genetic distance 2.251, standard deviation 1.493
Population of 20 members in 2 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
    15   40    11   -991.9    0.933    20
    19   40     9   -800.0    0.824    14
Total extinctions: 0
Generation time: 5.038 sec (5.172 ave